### Building a function for finding an overlap graph derived from multiple reads (or strings) 

#### Given a dataset of 10,000 reads, with no two reads have the same sequence of bases.
#### Questions: Find all pairs of reads with an exact suffix/prefix match of length at least 30. How many edges are in the graph?  In other words, how many distinct pairs of reads overlap? How many nodes in this graph have at least one outgoing edge? (In other words, how many reads have a suffix involved in an overlap)

#### Instructions given by the course materials are as followed:

Let every k-mer in the dataset have an associated Python set object, which starts out empty.
We use a Python dictionary to associate each k-mer with its corresponding set.
(1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer. If our read is GATTA and k=3, we would add GATTA to the set objects for GAT, ATT and TTA.
We do this for every read so that, at the end, eachset contains all reads containing the corresponding k-mer.
(2) Now, for each read a, we find all overlaps involving a suffix of a. To do this, we take a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding set) and call overlap(a, b, min_length=k) for each.

#### Download the dataset which includes 10,000 reads

In [ ]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

#### Read FASTQ file, return a list of reads

In [8]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename, 'r') as fh:
        while True:
            fh.readline()
            seq = fh.readline().rstrip()
            fh.readline()
            qual = fh.readline().rstrip()
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences  #, qualities

reads = readFastq('ERR266411_1.for_asm.fastq')

#### Functions built according to instructions

In [10]:
def all_kmers(read, k):
    """ Returns all k-mers of length k of a given read (string) """
    all_kmers = []
    for i in range(len(read) - k + 1):
        all_kmers.append(read[i: i+k])
    return all_kmers


def overlap(a, b, k):     # this function is given by the course
    """ Returns the length (of minimum k bases) of the overlap between read a and read b """
    start = 0
    while True:
        start = a.find(b[:k], start)
        if start == -1:
            return 0
        if b.startswith(a[start:]):
            return len(a) - start
        start += 1
        

def graphs(reads, k):
    """ Returns a list of pairs (or edges) of reads that overlap (at least k bases), and a list of nodes that
    have at least one outgoing edge (referred as tailnodes) """
    
    pairs = []
    tailnodes = set()
    suffixSet = {}
    
    for read in reads:
        kmers = all_kmers(read, k)
        for kmer in kmers:
            if not kmer in suffixSet.keys():
                suffixSet[kmer] = set()
            suffixSet[kmer].add(read)
    
    for read in reads:
        overlap_reads = suffixSet[read[-k:]]
        for each in overlap_reads:
            if each != read:
                if overlap(read, each, k) > 0:
                    pairs.append((read, each))
                    if not read in tailnodes:
                        tailnodes.add(read)
    return pairs, tailnodes

#### To answer the questions, simply run:

In [ ]:
len(pairs)

In [ ]:
len(tailnodes)